<a href="https://colab.research.google.com/github/sarjakpatel/CMPE_297_Advanced_Deep_Learning/blob/main/Assignment%205/CMPE_297_Assignment_5_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPE 297 Assignment 5 Part 1: continual learning using avalanche library

Avalanche and its dependencies can be installed using the following command. Avalanche runs on either CPU or CUDA GPU based on availability. Since Avalanche is built on top of PyTorch, a PyTorch environment has to be created.

Install Avalanche and its dependencies

In [1]:
!pip install git+https://github.com/ContinualAI/avalanche.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-r4lrkajc
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-r4lrkajc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 529 kB 10.1 MB/s 
     |████████████████████████████████| 427 kB 61.5 MB/s 
     |████████████████████████████████| 1.9 MB 51.9 MB/s 
     |████████████████████████████████| 532 kB 66.3 MB/s 
     |████████████████████████████████| 168 kB 56.7 MB/s 
     |████████████████████████████████| 182 kB 44.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 54.7 MB/s 
     |████████████████████████████████| 166 kB 43.7 MB/s 
     |████████████████████████████████| 162 kB 68.6 MB/s 
     |█

Create the environment by importing necessary libraries and modules.

In [2]:
import torch
# use CrossEntropyLoss
from torch.nn import CrossEntropyLoss
# use stochastic GD optimizer
from torch.optim import SGD
# import the PermutedMNIST dataset
from avalanche.benchmarks.classic import PermutedMNIST
# import the SimpleMLP dataset
from avalanche.models import SimpleMLP
# we will use Naive training strategy
from avalanche.training.supervised import Naive

Configure the device settings by checking for availability of CUDA GPU

In [3]:
# Configure the device settings
# check for CUDA GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Instantiate a simple multi-layer perceptron model**

In [ ]:
model = SimpleMLP(num_classes = 10)

**Prepare the benchmark dataset**

In [ ]:
# Benchmark instantiation
perm_mnist = PermutedMNIST(n_experiences=3)

**Split the dataset into train and test sets.**

In [4]:
# split stream into train and test set
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



Instantiate optimizer and loss functions.

In [5]:
# define optimizer and loss function for training
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

Devise a training strategy. Here, the Naive strategy is implemented.

In [6]:
# Instantiate Continual learning strategy
strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

**Mode Trainning and Evaluating the results**

In [7]:
results = []
# train and test epochs
for train_task in train_stream:
    # training
    strategy.train(train_task, num_workers=4)
    # evaluate the model and store the results
    results.append(strategy.eval(test_stream))

-- >> Start of training phase << --
0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 1875/1875 [01:17<00:00, 24.21it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4838
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8637
100%|██████████| 1875/1875 [01:17<00:00, 24.10it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2565
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9253
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:06<00:00, 46.49it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1830
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9471
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 313/313 [00:06<00:00, 45.94it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 2.7449
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.0593
-- S

In [10]:
print(results)

[{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.92525, 'Loss_Epoch/train_phase/train_stream/Task000': 0.25649423302710056, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9471, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.18301760343015194, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.0593, 'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 2.744920086669922, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002': 0.1218, 'Loss_Exp/eval_phase/test_stream/Task000/Exp002': 2.5689800910949705, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.37606666666666666, 'Loss_Stream/eval_phase/test_stream/Task000': 1.8323059270650148}, {'Top1_Acc_Epoch/train_phase/train_stream/Task000': 0.9332666666666667, 'Loss_Epoch/train_phase/train_stream/Task000': 0.22578439920445284, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.9344, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.2176342511549592, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.9555, 'L

**Avalanche provides the strategy of clean and simple code that makes it easy to use.**

## Reference: 



*   https://analyticsindiamag.com/avalanche-a-python-library-for-continual-learning/